In [1]:
from IPython.display import clear_output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd 
import subprocess #run R code from subprocess
import numpy as np
from os import listdir # get dir list
from os.path import isfile, join # get dir list


import concurrent.futures
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
def buildMetaBase(metabase, cols):
    
    meta_data = pd.read_csv(f'Metabases/Metabase_{metabase}.csv.zip', index_col='Dataset')[cols]
    meta_data.columns = ['Edge5','Edge20', 'HARF85','Hybrid', 'Edge15', 'OR3','GE9', 'AENN7',
       'HARF75', 'OR19','AENN11', 'AENN9','OR11', 'PRISM','GE3', 'GE7',
       'OR15', 'Dynamic','Edge10', 'HARF80','AENN3', 'HARF90','OR7', 'HARF70',
       'GE5', 'AENN5','GE11']
    rank = pd.DataFrame()
    # meta_data.drop('test_f1_macro_mean_AENN',axis=1, inplace=True)
    for i, a in enumerate(meta_data.index):
        rank = rank.append(meta_data.iloc[i].rank(method = 'dense', ascending = False))
    #get the best algorithm list
    rank_copy = rank.copy()
    # rank_copy.columns = ['Edge5','Edge20', 'HARF85','Hybrid', 'Edge15', 'OR3','GE9', 'AENN7',
    #    'HARF75', 'OR19','AENN11', 'AENN9','OR11', 'PRISM','GE3', 'GE7',
    #    'OR15', 'Dynamic','Edge10', 'HARF80','AENN3', 'HARF90','OR7', 'HARF70',
    #    'GE5', 'AENN5','GE11']
    
    

    best = rank.idxmin(axis=1)

    Ensamble = ['Edge5', 'Edge20', 'Edge15', 'OR3',
            'OR19', 'OR11', 'OR15',
            'Edge10','OR7']
    Voting = ['HARF70',
        'HARF75', 'HARF80',
        'HARF85', 'HARF90',
        'Hybrid', 'Dynamic']
    Distance = ['AENN11',
        'AENN3', 'AENN5',
        'AENN7', 'AENN9',
        'GE11', 'GE3',
        'GE5', 'GE7',
        'GE9', 'PRISM']
    ORBoosting = ['OR3','OR19', 'OR11', 'OR15','OR7']
    EdgeBoosting = ['Edge5', 'Edge10', 'Edge15', 'Edge20']
    Harf = ['HARF70','HARF75', 'HARF80','HARF85', 'HARF90',]
    hybrid = ['Hybrid']
    dynamic = ['Dynamic']
    Aenn = ['AENN11','AENN3', 'AENN5','AENN7', 'AENN9']
    Ge = ['GE11', 'GE3','GE5', 'GE7','GE9']
    Pris = ['PRISM']

    rank['BestAlgoType'] = best.replace(Voting, 'Voting').replace(Ensamble, 'Ensamble').replace(Distance, 'Distance')
    Ensamble_rank = rank[Ensamble]
    rank['BestEnsambleAlgo'] = Ensamble_rank.idxmin(axis=1).replace(ORBoosting, 'ORBoost').replace(EdgeBoosting, 'Edge')
    Distance_rank = rank[Distance]
    rank['BestDistanceAlgo'] = Distance_rank.idxmin(axis=1).replace(Ge, 'GE').replace(Pris, 'PRISM').replace(Aenn, 'AENN')
    Voting_rank = rank[Voting]
    rank['BestVotingAlgo'] = Voting_rank.idxmin(axis=1).replace(Harf, 'HARF').replace(hybrid, 'Hybrid').replace(dynamic, 'Dynamic')
    Aenn_rank = rank[Aenn]
    rank['BestAenn'] = Aenn_rank.idxmin(axis=1).replace(Aenn[0], 'AENN11').replace(Aenn[1], 'AENN3').replace(Aenn[2], 'AENN5').replace(Aenn[3], 'AENN7').replace(Aenn[4], 'AENN9')
    Ge_rank = rank[Ge]
    rank['BestGE'] = Ge_rank.idxmin(axis=1).replace(Ge[0], 'GE11').replace(Ge[1], 'GE3').replace(Ge[2], 'GE5').replace(Ge[3], 'GE7').replace(Ge[4], 'GE9')
    OR_rank = rank[ORBoosting]
    rank['BestOR'] = OR_rank.idxmin(axis=1).replace(ORBoosting[0], 'OR3').replace(ORBoosting[1], 'OR19').replace(ORBoosting[2], 'OR11').replace(ORBoosting[3], 'OR15').replace(ORBoosting[4], 'OR7')
    Edge_rank = rank[EdgeBoosting]
    rank['BestEdge'] = Edge_rank.idxmin(axis=1).replace(EdgeBoosting[0], 'Edge5').replace(EdgeBoosting[1], 'Edge10').replace(EdgeBoosting[2], 'Edge15').replace(EdgeBoosting[3], 'Edge20')
    HARF_rank = rank[Harf]
    rank['BestHARF'] = HARF_rank.idxmin(axis=1).replace(Harf[0], 'HARF70').replace(Harf[1], 'HARF75').replace(Harf[2], 'HARF80').replace(Harf[3], 'HARF85').replace(Harf[4], 'HARF90')
    # best.replace(Ensamble, 1)
    # best.replace(Distance, -1)
    # best
    rank
    multiple_classifiers_metaBase = pd.read_csv('Metabases/Metabase_DT.csv.zip', index_col='Dataset').iloc[:,:97]
    multiple_classifiers_metaBase['BestType'] = best.replace(Voting, 'Voting').replace(Ensamble, 'Ensamble').replace(Distance, 'Distance')

    multiple_classifiers_metaBase['BestEnsambleAlgo'] = Ensamble_rank.idxmin(axis=1).replace(ORBoosting, 'ORBoost').replace(EdgeBoosting, 'Edge')
    multiple_classifiers_metaBase['BestDistanceAlgo'] = Distance_rank.idxmin(axis=1).replace(Ge, 'GE').replace(Pris, 'Prism').replace(Aenn, 'AENN')
    multiple_classifiers_metaBase['BestVotingAlgo'] = Voting_rank.idxmin(axis=1).replace(Harf, 'HARF').replace(hybrid, 'Hybrid').replace(dynamic, 'Dynamic')

    multiple_classifiers_metaBase['BestAenn'] = rank['BestAenn']
    multiple_classifiers_metaBase['BestGE'] = rank['BestGE']
    multiple_classifiers_metaBase['BestOR'] = rank['BestOR']
    multiple_classifiers_metaBase['BestEdge'] = rank['BestEdge']
    multiple_classifiers_metaBase['BestHARF'] = rank['BestHARF']

    return multiple_classifiers_metaBase

In [3]:
metabasesTypes = ['DT', 'KNN', 'RF']
cols = pd.read_csv(f'Metabases/Metabase_DT.csv.zip', index_col='Dataset').iloc[:,97:].columns
for metabaseType in metabasesTypes:
    df = buildMetaBase(metabaseType, cols)
    df.to_csv(f'Metabases/metabases_Multiple_Classifiers/Metabase_{metabaseType}.csv.zip')
    display(df)

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,cat_to_num,class_conc.mean,...,worst_node.sd,BestType,BestEnsambleAlgo,BestDistanceAlgo,BestVotingAlgo,BestAenn,BestGE,BestOR,BestEdge,BestHARF
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,8.331300e-02,0.062438,2.999999,0.000006,0.1000,0.878333,0.076965,0.688118,0.0,0.012346,...,0.003727,Distance,ORBoost,AENN,Dynamic,AENN11,GE9,OR15,Edge20,HARF70
1004_synthetic_control_10_1.csv.gz,1.005420e-01,0.057744,2.999999,0.000006,0.1000,0.885000,0.084451,0.707928,0.0,0.012193,...,0.000000,Voting,Edge,AENN,Dynamic,AENN11,GE11,OR3,Edge20,HARF75
1004_synthetic_control_10_2.csv.gz,8.331762e-02,0.067932,2.999999,0.000006,0.1000,0.875000,0.087995,0.729191,0.0,0.012570,...,0.004564,Voting,ORBoost,AENN,HARF,AENN11,GE11,OR15,Edge20,HARF70
1004_synthetic_control_10_3.csv.gz,6.843893e-02,0.060531,2.999999,0.000006,0.1000,0.861667,0.078969,0.718893,0.0,0.012131,...,0.014434,Voting,ORBoost,AENN,Hybrid,AENN11,GE9,OR3,Edge15,HARF75
1004_synthetic_control_10_4.csv.gz,6.317691e-02,0.059250,2.999999,0.000006,0.1000,0.870000,0.083458,0.718633,0.0,0.012191,...,0.000000,Ensamble,ORBoost,GE,HARF,AENN11,GE7,OR11,Edge20,HARF80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.512000,0.043451,0.709977,0.0,0.032749,...,0.010431,Distance,ORBoost,AENN,Dynamic,AENN5,GE11,OR19,Edge20,HARF85
997_balance-scale_5_6.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.507200,0.040635,0.711547,0.0,0.032597,...,0.060926,Distance,Edge,AENN,Dynamic,AENN3,GE3,OR7,Edge20,HARF70
997_balance-scale_5_7.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.504000,0.035327,0.729005,0.0,0.034303,...,0.029611,Distance,Edge,AENN,Dynamic,AENN9,GE7,OR11,Edge20,HARF75


,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,cat_to_num,class_conc.mean,...,worst_node.sd,BestType,BestEnsambleAlgo,BestDistanceAlgo,BestVotingAlgo,BestAenn,BestGE,BestOR,BestEdge,BestHARF
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,8.331300e-02,0.062438,2.999999,0.000006,0.1000,0.878333,0.076965,0.688118,0.0,0.012346,...,0.003727,Voting,ORBoost,AENN,Hybrid,AENN11,GE11,OR3,Edge20,HARF70
1004_synthetic_control_10_1.csv.gz,1.005420e-01,0.057744,2.999999,0.000006,0.1000,0.885000,0.084451,0.707928,0.0,0.012193,...,0.000000,Ensamble,ORBoost,AENN,Dynamic,AENN11,GE11,OR3,Edge20,HARF70
1004_synthetic_control_10_2.csv.gz,8.331762e-02,0.067932,2.999999,0.000006,0.1000,0.875000,0.087995,0.729191,0.0,0.012570,...,0.004564,Voting,ORBoost,GE,HARF,AENN11,GE9,OR3,Edge20,HARF70
1004_synthetic_control_10_3.csv.gz,6.843893e-02,0.060531,2.999999,0.000006,0.1000,0.861667,0.078969,0.718893,0.0,0.012131,...,0.014434,Voting,ORBoost,GE,Hybrid,AENN11,GE11,OR3,Edge15,HARF70
1004_synthetic_control_10_4.csv.gz,6.317691e-02,0.059250,2.999999,0.000006,0.1000,0.870000,0.083458,0.718633,0.0,0.012191,...,0.000000,Voting,ORBoost,AENN,Hybrid,AENN11,GE11,OR3,Edge15,HARF70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.512000,0.043451,0.709977,0.0,0.032749,...,0.010431,Distance,Edge,AENN,Dynamic,AENN9,GE7,OR3,Edge20,HARF70
997_balance-scale_5_6.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.507200,0.040635,0.711547,0.0,0.032597,...,0.060926,Distance,ORBoost,AENN,Dynamic,AENN11,GE11,OR3,Edge20,HARF70
997_balance-scale_5_7.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.504000,0.035327,0.729005,0.0,0.034303,...,0.029611,Distance,Edge,AENN,Dynamic,AENN11,GE9,OR3,Edge20,HARF75


,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,cat_to_num,class_conc.mean,...,worst_node.sd,BestType,BestEnsambleAlgo,BestDistanceAlgo,BestVotingAlgo,BestAenn,BestGE,BestOR,BestEdge,BestHARF
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,8.331300e-02,0.062438,2.999999,0.000006,0.1000,0.878333,0.076965,0.688118,0.0,0.012346,...,0.003727,Distance,ORBoost,AENN,Dynamic,AENN11,GE7,OR11,Edge20,HARF70
1004_synthetic_control_10_1.csv.gz,1.005420e-01,0.057744,2.999999,0.000006,0.1000,0.885000,0.084451,0.707928,0.0,0.012193,...,0.000000,Distance,Edge,AENN,Dynamic,AENN11,GE5,OR11,Edge20,HARF75
1004_synthetic_control_10_2.csv.gz,8.331762e-02,0.067932,2.999999,0.000006,0.1000,0.875000,0.087995,0.729191,0.0,0.012570,...,0.004564,Ensamble,ORBoost,AENN,HARF,AENN11,GE5,OR3,Edge20,HARF70
1004_synthetic_control_10_3.csv.gz,6.843893e-02,0.060531,2.999999,0.000006,0.1000,0.861667,0.078969,0.718893,0.0,0.012131,...,0.014434,Distance,ORBoost,AENN,Dynamic,AENN11,GE7,OR3,Edge20,HARF80
1004_synthetic_control_10_4.csv.gz,6.317691e-02,0.059250,2.999999,0.000006,0.1000,0.870000,0.083458,0.718633,0.0,0.012191,...,0.000000,Ensamble,ORBoost,AENN,Dynamic,AENN11,GE5,OR7,Edge20,HARF70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.512000,0.043451,0.709977,0.0,0.032749,...,0.010431,Distance,ORBoost,AENN,Dynamic,AENN11,GE3,OR3,Edge20,HARF70
997_balance-scale_5_6.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.507200,0.040635,0.711547,0.0,0.032597,...,0.060926,Distance,Edge,AENN,Dynamic,AENN7,GE3,OR7,Edge20,HARF70
997_balance-scale_5_7.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.504000,0.035327,0.729005,0.0,0.034303,...,0.029611,Distance,Edge,AENN,Dynamic,AENN11,GE3,OR3,Edge20,HARF70


In [3]:
metabasesTypes = ['SVM']
cols = pd.read_csv(f'Metabases/Metabase_DT.csv.zip', index_col='Dataset').iloc[:,97:].columns
for metabaseType in metabasesTypes:
    df = buildMetaBase(metabaseType, cols)
    df.to_csv(f'Metabases/metabases_Multiple_Classifiers/Metabase_{metabaseType}.csv.zip')
    display(df)

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,cat_to_num,class_conc.mean,...,worst_node.sd,BestType,BestEnsambleAlgo,BestDistanceAlgo,BestVotingAlgo,BestAenn,BestGE,BestOR,BestEdge,BestHARF
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,8.331300e-02,0.062438,2.999999,0.000006,0.1000,0.878333,0.076965,0.688118,0.0,0.012346,...,0.003727,Voting,ORBoost,AENN,Hybrid,AENN11,GE11,OR3,Edge20,HARF70
1004_synthetic_control_10_1.csv.gz,1.005420e-01,0.057744,2.999999,0.000006,0.1000,0.885000,0.084451,0.707928,0.0,0.012193,...,0.000000,Ensamble,ORBoost,AENN,Dynamic,AENN11,GE11,OR3,Edge20,HARF70
1004_synthetic_control_10_2.csv.gz,8.331762e-02,0.067932,2.999999,0.000006,0.1000,0.875000,0.087995,0.729191,0.0,0.012570,...,0.004564,Voting,ORBoost,GE,HARF,AENN11,GE9,OR3,Edge20,HARF70
1004_synthetic_control_10_3.csv.gz,6.843893e-02,0.060531,2.999999,0.000006,0.1000,0.861667,0.078969,0.718893,0.0,0.012131,...,0.014434,Voting,ORBoost,AENN,Hybrid,AENN11,GE11,OR3,Edge15,HARF70
1004_synthetic_control_10_4.csv.gz,6.317691e-02,0.059250,2.999999,0.000006,0.1000,0.870000,0.083458,0.718633,0.0,0.012191,...,0.000000,Voting,ORBoost,AENN,Hybrid,AENN11,GE11,OR3,Edge15,HARF70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.512000,0.043451,0.709977,0.0,0.032749,...,0.010431,Distance,ORBoost,AENN,Dynamic,AENN11,GE7,OR3,Edge20,HARF70
997_balance-scale_5_6.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.507200,0.040635,0.711547,0.0,0.032597,...,0.060926,Distance,ORBoost,AENN,Dynamic,AENN9,GE11,OR7,Edge20,HARF70
997_balance-scale_5_7.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.504000,0.035327,0.729005,0.0,0.034303,...,0.029611,Distance,ORBoost,AENN,Dynamic,AENN11,GE11,OR3,Edge20,HARF70
